## Setting up a SWAT Connection for Python

In [5]:
import os
import swat

In [6]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = R"C:\SAS Viya\CAS_demo_ex_cert.crt"

In [7]:
conn = swat.CAS(
  hostname = "sas-cas-server-default-bin-demo.uksouth.cloudapp.azure.com",
  port=5570,
  authinfo=R"C:\SAS Viya\.authinfo"
)

In [ ]:
conn.serverstatus()

## Create a reference to in-memory tables

In [ ]:
cirrhosis = conn.CASTable("cirrhosis", caslib="public")

## Use pandas code and functions on the CAS tables as if they were standard dataframes

These will be translated into CAS actions, and alter the underlying table in CAS

In [ ]:
# Get head data from table, methods similar to pandas
cirrhosis.head()

In [ ]:
# Work with the tables as if they were pandas dataframes

cirrhosis["Bilirubin_high"] = cirrhosis["Bilirubin"] > 1.2

## Use the full capabilities of CAS

Can load in Action Sets to perform any CAS action, i.e. powerful in-memory & distributed analysis

In [ ]:
conn.loadactionset("regression")

In [ ]:
output = cirrhosis.glm(
    target = "Platelets",
    inputs = ["Stage", "Status", "Drug", "Bilirubin_high", "Albumin", "Copper"],
    nominals = ["Stage"],
    output = {
        "casOut": {"name": "PlateletsPrediction", "replace": True},
        "copyvars": "all",
        "pred": "PredictedPlatelets",
        "resid": "ResidualPlatelets",
    }
)


In [ ]:
result = conn.CASTable("PlateletsPrediction")

In [ ]:
result.head()

In [ ]:
result.plot.scatter("PredictedPlatelets", "Platelets")